In [ ]:
from rich import print
import logging

logging.basicConfig(level=INFO)

# Combining data + CLI

Here we show how to combine multiple datasets, export them to recipes, and run
these from the command line.

## Combining datasets

In the previous section we've seen that one of the main goals of springtime is
to harmonize datasets from different sources, such that we can easily combine
them. Here, we walk through an example with data from PEP725 and EOBS to show
how this is done.

We start with basic observations from PEP725.


In [ ]:
from springtime.datasets import PEP725Phenor
from springtime.utils import germany

pep725 = PEP725Phenor(
    species="Syringa vulgaris",
    years=[2000, 2002],
    area=germany,
)

df_pep725 = pep725.load()

Next, we want to find matching meteo data from E-OBS:


In [ ]:
from springtime.datasets import EOBS
from springtime.utils import PointsFromOther

eobs = EOBS(
    area=germany,
    years=["2000", "2002"],
    variables=["mean_temperature", "minimum_temperature"],
    resample={"frequency": "M", "operator": "mean"},
    points=PointsFromOther(source="pep725"),
)

Notice that we're using a special object called `PointsFromOther`. This helper object can retrieve the records from our pep725 dataset, and use those to subselect the E-OBS data. To this end, we call the `get_points` method with the pep725 dataframe as input. This seems convoluted, but as we will see later, it will help to write very succinct recipes.


In [ ]:
eobs.points.get_points(df_pep725)
df_eobs = eobs.load()

Now, we're ready to join our dataframes.


In [18]:
from springtime.utils import join_dataframes

join_dataframes([df_pep725, df_eobs])

day  mean_temperature|31  mean_temperature|59  \
year geometry                                                                   
2000 POINT (10.00000 49.48330)  129             0.323548                  NaN   
     POINT (10.00000 50.85000)  120             0.943226                  NaN   
     POINT (10.00000 51.71670)  116             1.694194                  NaN   
     POINT (10.00000 52.10000)  120             2.531935                  NaN   
     POINT (10.00000 53.08330)  121             2.119677                  NaN   
...                             ...                  ...                  ...   
2002 POINT (9.96667 50.15000)   120             0.033871             5.121071   
     POINT (9.96667 50.95000)   131             0.667097             5.050714   
     POINT (9.96667 52.81670)   131             2.531290             4.775357   
     POINT (9.98333 49.76670)   118             0.221613             5.803572   
     POINT (9.98333 53.36670)   127             3.214839             5.342857   

                                mean_temperature|60  mean_temperature|90  \
year geometry                                                              
2000 POINT (10.00000 49.48330)             3.664483                  NaN   
     POINT (10.00000 50.85000)             3.795517                  NaN   
     POINT (10.00000 51.71670)             4.053448                  NaN   
     POINT (10.00000 52.10000)             4.937242                  NaN   
     POINT (10.00000 53.08330)             3.988276                  NaN   
...                                             ...                  ...   
2002 POINT (9.96667 50.15000)                   NaN             5.777419   
     POINT (9.96667 50.95000)                   NaN             4.759677   
     POINT (9.96667 52.81670)                   NaN             4.818065   
     POINT (9.98333 49.76670)                   NaN             6.415483   
     POINT (9.98333 53.36670)                   NaN             5.029356   

                                mean_temperature|91  mean_temperature|120  \
year geometry                                                               
2000 POINT (10.00000 49.48330)             5.358709                   NaN   
     POINT (10.00000 50.85000)             5.660645                   NaN   
     POINT (10.00000 51.71670)             5.399354                   NaN   
     POINT (10.00000 52.10000)             5.771289                   NaN   
     POINT (10.00000 53.08330)             4.812258                   NaN   
...                                             ...                   ...   
2002 POINT (9.96667 50.15000)                   NaN              8.711332   
     POINT (9.96667 50.95000)                   NaN              7.534667   
     POINT (9.96667 52.81670)                   NaN              7.735334   
     POINT (9.98333 49.76670)                   NaN              9.338666   
     POINT (9.98333 53.36670)                   NaN              8.083667   

                                mean_temperature|121  mean_temperature|151  \
year geometry                                                                
2000 POINT (10.00000 49.48330)              9.966999                   NaN   
     POINT (10.00000 50.85000)             10.001336                   NaN   
     POINT (10.00000 51.71670)             10.563000                   NaN   
     POINT (10.00000 52.10000)             10.993333                   NaN   
     POINT (10.00000 53.08330)              9.906999                   NaN   
...                                              ...                   ...   
2002 POINT (9.96667 50.15000)                    NaN             13.648706   
     POINT (9.96667 50.95000)                    NaN             13.160967   
     POINT (9.96667 52.81670)                    NaN             14.055484   
     POINT (9.98333 49.76670)                    NaN             14.178065   
     POINT (9.98333 53.36670)                    Na

## From datasets to workflow

We've already had a sneak preview of yaml for indivual datasets. We can also combine the two datasets into what we call a "workflow".


In [ ]:
from springtime.main import Workflow, Session

workflow = Workflow(datasets={"pep725": pep725, "eobs": eobs})
print(workflow)

To execute the workflow we first create a session and set the log level to info. This will provide a bit more info about the progress and the data will automatically be stored in a dedicated output folder.


In [21]:
session = Session()
workflow.execute(session)

INFO:springtime.main:Dataset pep725 loaded with 4723 rows
INFO:springtime.datasets.meteo.eobs:Locating data
INFO:springtime.datasets.meteo.eobs:Looking for variable mean_temperature in period 2000-2002...
INFO:springtime.datasets.meteo.eobs:Found /home/peter/.cache/springtime/e-obs/tg_ens_mean_0.1deg_reg_1995-2010_v26.0e.nc
INFO:springtime.datasets.meteo.eobs:Looking for variable minimum_temperature in period 2000-2002...
INFO:springtime.datasets.meteo.eobs:Found /home/peter/.cache/springtime/e-obs/tn_ens_mean_0.1deg_reg_1995-2010_v26.0e.nc


INFO:springtime.main:Dataset eobs loaded with 4723 rows
INFO:springtime.main:Datasets joined to shape: (4729, 47)
INFO:springtime.main:Data saved to: /tmp/output/data.csv


Workflows can also be represented in recipes.


In [20]:
recipe = workflow.to_recipe()
print(recipe)

datasets:
  pep725:
    dataset: PEP725Phenor
    years:
    - 2000
    - 2002
    credential_file: /home/peter/.config/springtime/pep725_credentials.txt
    species: Syringa vulgaris
    phenophase: null
    include_cols:
    - year
    - geometry
    - day
    area:
      name: Germany
      bbox:
      - 5.98865807458
      - 47.3024876979
      - 15.0169958839
      - 54.983104153
  eobs:
    dataset: E-OBS
    years:
    - 2000
    - 2002
    product_type: ensemble_mean
    variables:
    - mean_temperature
    - minimum_temperature
    grid_resolution: 0.1deg
    version: 26.0e
    points:
      source: pep725
    keep_grid_location: false
    area:
      name: Germany
      bbox:
      - 5.98865807458
      - 47.3024876979
      - 15.0169958839
      - 54.983104153
    minimize_cache: false
    resample: {}

## Springtime's command line interface

Springtime recipes can also be executed from the command line. If we saved the recipe above as `recipe_pep_eobs.yaml`, we could execute it as follows:

```bash
springtime recipe_pep_eobs.yaml
```

The springtime command is available after (pip) installing springtime in your python environment.

Executing recipes from the command line makes it easy to automate tasks or submit them as long-running compute jobs.
